## Exercises Ensemble models

1.- Use dataset Autoprice to train several regression models. You can choose your favourites among **Bagging, Stacking and Boosting** models. Train at least 2 regressions models with the best hyperparameters (You can find the best ones using GridSearch)

In [133]:
from sklearn.ensemble import StackingClassifier, StackingRegressor, RandomForestClassifier, RandomForestRegressor

In [135]:
autoprice = pd.read_csv('drive/MyDrive/Ironhack/Data/autoprice.csv', sep = ';')
autoprice.head()

,normalized-losses,wheel-base,length,width,height,curb-weight,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,95,109.1,188.8,68.9,55.5,3062,3.78,3.15,9.5,114,5400,19,25,22.625
1,115,98.8,177.8,66.5,55.5,2425,3.39,3.39,8.6,84,4800,26,32,11.245
2,104,99.1,186.6,66.5,56.1,2758,3.54,3.07,9.3,110,5250,21,28,15.510
3,161,93.7,157.3,64.4,50.8,1918,2.97,3.23,9.4,68,5500,37,41,5.389
4,78,96.5,157.1,63.9,58.3,2024,2.92,3.41,9.2,76,6000,30,34,7.295


In [136]:
auto_X_train, auto_X_test, auto_y_train, auto_y_test = train_test_split(autoprice[['normalized-losses','wheel-base','length','height','curb-weight','bore',
                                              'stroke','compression-ratio','horsepower','peak-rpm','city-mpg','highway-mpg']],
                                                    autoprice['price'], train_size = 0.8, random_state = 0)
auto_X_train.head()

,normalized-losses,wheel-base,length,height,curb-weight,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg
211,103,94.5,170.2,53.5,2037,3.15,3.29,9.40,69,5200,31,37
500,81,95.7,169.7,59.1,2290,3.05,3.03,9.00,62,4800,27,32
497,93,106.7,187.5,54.9,3495,3.58,3.64,21.50,123,4350,22,25
10,188,101.2,176.8,54.3,2765,3.31,3.19,9.00,121,4250,21,28
475,150,99.1,186.6,56.1,2658,3.54,3.07,9.31,110,5250,21,28


In [137]:
column_to_keep_train = auto_X_train['city-mpg']
auto_X_train = auto_X_train.drop(columns=['city-mpg'])
original_index_train = auto_X_train.index

column_to_keep_test = auto_X_test['city-mpg']
auto_X_test = auto_X_test.drop(columns=['city-mpg'])
original_index_test = auto_X_test.index

scaler = StandardScaler()
auto_scaler = scaler.fit(auto_X_train)
train_auto_scaler = auto_scaler.transform(auto_X_train)
test_auto_scaler = auto_scaler.transform(auto_X_test)

# Crear un nuevo DataFrame con las características estandarizadas
auto_X_train_auto_scaler = pd.DataFrame(train_auto_scaler, columns=auto_X_train.columns, index = original_index_train)
auto_X_test_auto_scaler = pd.DataFrame(test_auto_scaler, columns=auto_X_test.columns, index = original_index_test)

# Concatenar la columna no estandarizada de nuevo al DataFrame
auto_X_train_auto_scaler = pd.concat([auto_X_train_auto_scaler, column_to_keep_train], axis=1)
auto_X_test_auto_scaler = pd.concat([auto_X_test_auto_scaler, column_to_keep_test], axis=1)
auto_X_train_auto_scaler.head(3)

,normalized-losses,wheel-base,length,height,curb-weight,bore,stroke,compression-ratio,horsepower,peak-rpm,highway-mpg,city-mpg
211,-0.492672,-0.739672,-0.160749,-0.211760,-0.878340,-0.591682,0.225063,-0.220402,-0.898631,0.198002,0.757482,31
500,-1.120355,-0.501370,-0.205145,2.328946,-0.321082,-0.958992,-0.646542,-0.319283,-1.153401,-0.637417,-0.049182,27
497,-0.777982,1.683066,1.375356,0.423416,2.333055,0.987749,1.398377,2.770724,1.066740,-1.577262,-1.178512,22


In [138]:
RF_Reg = RandomForestRegressor()

grid = dict()
grid['n_estimators'] = [10, 50] # number of trees
grid['criterion'] = ['squared_error','absolute_error']



# define the evaluation procedure
cv = RepeatedKFold(n_splits = 5, n_repeats = 3, random_state = 1)

# define the grid search procedure
grid_search = GridSearchCV(estimator = RF_Reg, param_grid = grid, n_jobs = -1, cv = cv, scoring = 'neg_mean_absolute_error')

# execute the grid search
grid_result = grid_search.fit(auto_X_train_auto_scaler, auto_y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# summarize all scores that were evaluated
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.406460 using {'criterion': 'squared_error', 'n_estimators': 50}
-0.432145 (0.109589) with: {'criterion': 'squared_error', 'n_estimators': 10}
-0.406460 (0.083315) with: {'criterion': 'squared_error', 'n_estimators': 50}
-0.441394 (0.095318) with: {'criterion': 'absolute_error', 'n_estimators': 10}
-0.431687 (0.118824) with: {'criterion': 'absolute_error', 'n_estimators': 50}


In [139]:
RF_Reg = RandomForestRegressor(criterion = 'squared_error', n_estimators = 500)
RF_Reg.fit(auto_X_train_auto_scaler, auto_y_train)

RandomForestRegressor(n_estimators=500)

In [140]:
auto_X_test_auto_scaler['RF_Reg'] = RF_Reg.predict(auto_X_test_auto_scaler)
auto_X_train_auto_scaler['RF_Reg'] = RF_Reg.predict(auto_X_train_auto_scaler)

In [141]:
print("MAE: ", metrics.mean_absolute_error(auto_y_train, auto_X_train_auto_scaler['RF_Reg']).round(4))
print("MSE: ", metrics.mean_squared_error(auto_y_train, auto_X_train_auto_scaler['RF_Reg']).round(4))
print("RMSE: ", np.sqrt(metrics.mean_squared_error(auto_y_train, auto_X_train_auto_scaler['RF_Reg'])).round(4))
print("MAPE: ", metrics.mean_absolute_percentage_error(auto_y_train, auto_X_train_auto_scaler['RF_Reg']).round(4))
print("R2: ", metrics.r2_score(auto_y_train, auto_X_train_auto_scaler['RF_Reg']).round(4))

MAE:  0.1133
MSE:  0.076
RMSE:  0.2757
MAPE:  0.0092
R2:  0.9976


In [142]:
print("MAE: ", metrics.mean_absolute_error(auto_y_test, auto_X_test_auto_scaler['RF_Reg']).round(4))
print("MSE: ", metrics.mean_squared_error(auto_y_test, auto_X_test_auto_scaler['RF_Reg']).round(4))
print("RMSE: ", np.sqrt(metrics.mean_squared_error(auto_y_test, auto_X_test_auto_scaler['RF_Reg'])).round(4))
print("MAPE: ", metrics.mean_absolute_percentage_error(auto_y_test, auto_X_test_auto_scaler['RF_Reg']).round(4))
print("R2: ", metrics.r2_score(auto_y_test, auto_X_test_auto_scaler['RF_Reg']).round(4))

MAE:  0.2392
MSE:  0.3151
RMSE:  0.5614
MAPE:  0.0199
R2:  0.9909


In [143]:
#Otro modelo

In [144]:
level0 = list()
level0.append(('lr', LinearRegression()))
level0.append(('RF', RandomForestRegressor()))
level0.append(('svr', SVR()))

level1 = LinearRegression()

# define the stacking ensemble
St_reg = StackingRegressor(estimators = level0, final_estimator = level1)

# fit the model on all available data
St_reg.fit(auto_X_train_auto_scaler, auto_y_train)

StackingRegressor(estimators=[('lr', LinearRegression()),
                              ('RF', RandomForestRegressor()), ('svr', SVR())],
                  final_estimator=LinearRegression())

In [145]:
auto_X_test_auto_scaler['St_reg'] = St_reg.predict(auto_X_test_auto_scaler)
auto_X_train_auto_scaler['St_reg'] = St_reg.predict(auto_X_train_auto_scaler)

In [146]:
print("MAE: ", metrics.mean_absolute_error(auto_y_train, auto_X_train_auto_scaler['St_reg']).round(4))
print("MSE: ", metrics.mean_squared_error(auto_y_train, auto_X_train_auto_scaler['St_reg']).round(4))
print("RMSE: ", np.sqrt(metrics.mean_squared_error(auto_y_train, auto_X_train_auto_scaler['St_reg'])).round(4))
print("MAPE: ", metrics.mean_absolute_percentage_error(auto_y_train, auto_X_train_auto_scaler['St_reg']).round(4))
print("R2: ", metrics.r2_score(auto_y_train, auto_X_train_auto_scaler['St_reg']).round(4))

MAE:  0.1473
MSE:  0.0753
RMSE:  0.2744
MAPE:  0.0129
R2:  0.9976


In [147]:
print("MAE: ", metrics.mean_absolute_error(auto_y_test, auto_X_test_auto_scaler['St_reg']).round(4))
print("MSE: ", metrics.mean_squared_error(auto_y_test, auto_X_test_auto_scaler['St_reg']).round(4))
print("RMSE: ", np.sqrt(metrics.mean_squared_error(auto_y_test, auto_X_test_auto_scaler['St_reg'])).round(4))
print("MAPE: ", metrics.mean_absolute_percentage_error(auto_y_test, auto_X_test_auto_scaler['St_reg']).round(4))
print("R2: ", metrics.r2_score(auto_y_test, auto_X_test_auto_scaler['St_reg']).round(4))


MAE:  0.247
MSE:  0.3257
RMSE:  0.5707
MAPE:  0.0215
R2:  0.9906


2.- Use Breast-cancer dataset to train several Classification models. You can choose your favourites among Bagging, Stacking and Boosting models. Train at least 2 classification models with the best hyperparameters (You can find the best ones using GridSearch)

In [88]:
cancer = pd.read_csv('drive/MyDrive/Ironhack/Data/breast-cancer.csv', sep = ';')
cancer.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')

In [108]:
x=cancer[['radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst']]
y=cancer["diagnosis"]

In [109]:
x.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [110]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(y)
y=le.transform(y)

In [111]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(x)

In [113]:
X_train,X_test,y_train,y_test=train_test_split(X,y, train_size = 0.8, random_state = 0)

In [115]:
from sklearn.ensemble import StackingClassifier, StackingRegressor, RandomForestClassifier
RF_Cl = RandomForestClassifier()

# define the grid of values to search
grid = dict()
grid['n_estimators'] = [100, 500]
grid['criterion'] = ['gini','entropy']

# define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)

# define the grid search procedure
grid_search = GridSearchCV(estimator=RF_Cl, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy')

# execute the grid search
grid_result = grid_search.fit(X_train, y_train)

# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# summarize all scores that were evaluated
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.959707 using {'criterion': 'gini', 'n_estimators': 100}
0.959707 (0.018678) with: {'criterion': 'gini', 'n_estimators': 100}
0.958974 (0.017704) with: {'criterion': 'gini', 'n_estimators': 500}
0.953114 (0.011676) with: {'criterion': 'entropy', 'n_estimators': 100}
0.954579 (0.014942) with: {'criterion': 'entropy', 'n_estimators': 500}


In [116]:
RF_Cl = RandomForestClassifier(criterion = 'gini', n_estimators = 100)
RF_Cl.fit(X_train, y_train)

RandomForestClassifier()

In [118]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix, roc_curve, roc_auc_score
from tabulate import tabulate

In [125]:
print("Train set score (Accuracy) =", RF_Cl.score(X_train, y_train).round(4))
print("Test set score (Accuracy) =", RF_Cl.score(X_test, y_test).round(4))

conf_mat = confusion_matrix(y_test, RF_Cl.predict(X_test))
print(tabulate(conf_mat,headers = ['Cancer No','Cancer Yes'], showindex = ['Cancer No','Cancer Yes'],
               tablefmt = 'fancy_grid'))

print(classification_report(y_test, RF_Cl.predict(X_test)))

Train set score (Accuracy) = 1.0
Test set score (Accuracy) = 0.9737
╒════════════╤═════════════╤══════════════╕
│            │   Cancer No │   Cancer Yes │
╞════════════╪═════════════╪══════════════╡
│ Cancer No  │          66 │            1 │
├────────────┼─────────────┼──────────────┤
│ Cancer Yes │           2 │           45 │
╘════════════╧═════════════╧══════════════╛
              precision    recall  f1-score   support

           0       0.97      0.99      0.98        67
           1       0.98      0.96      0.97        47

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114



In [126]:
#Otro modelo
from sklearn.linear_model import LogisticRegression, LinearRegression

In [127]:
level0 = list()
level0.append(('lr', LogisticRegression()))
level0.append(('RF', RandomForestClassifier()))
level0.append(('svc', SVC()))

level1 = LogisticRegression()

# define the stacking ensemble
St_Cl = StackingClassifier(estimators = level0, final_estimator = level1)

# fit the model on all available data
St_Cl.fit(X_train, y_train)

StackingClassifier(estimators=[('lr', LogisticRegression()),
                               ('RF', RandomForestClassifier()),
                               ('svc', SVC())],
                   final_estimator=LogisticRegression())

In [129]:
print("Train set score (Accuracy) =", St_Cl.score(X_train, y_train).round(4))
print("Test set score (Accuracy) =", St_Cl.score(X_test, y_test).round(4))

conf_mat = confusion_matrix(y_test, St_Cl.predict(X_test))
print(tabulate(conf_mat,headers = ['Cancer No','Cancer Yes'], showindex = ['Cancer No','Cancer Yes'],
               tablefmt = 'fancy_grid'))

print(classification_report(y_test, St_Cl.predict(X_test)))

Train set score (Accuracy) = 0.989
Test set score (Accuracy) = 0.9649
╒════════════╤═════════════╤══════════════╕
│            │   Cancer No │   Cancer Yes │
╞════════════╪═════════════╪══════════════╡
│ Cancer No  │          65 │            2 │
├────────────┼─────────────┼──────────────┤
│ Cancer Yes │           2 │           45 │
╘════════════╧═════════════╧══════════════╛
              precision    recall  f1-score   support

           0       0.97      0.97      0.97        67
           1       0.96      0.96      0.96        47

    accuracy                           0.96       114
   macro avg       0.96      0.96      0.96       114
weighted avg       0.96      0.96      0.96       114



## Exercises SVM_Models

3.- Using the code and the datasets of the theory notebooks (SVM_Models.ipynb) and **Grid Search**, find the best hyperparameters for the SVM and for the SVR models

In [50]:
# Data Manipulation
import pandas as pd # for data manipulation
import numpy as np # for data manipulation

# Sklearn
from sklearn.linear_model import LinearRegression # for building a linear regression model
from sklearn.svm import SVR, SVC # for building SVR model
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.model_selection import RepeatedKFold, RepeatedStratifiedKFold, GridSearchCV, cross_val_score
from sklearn.model_selection import train_test_split

# Visualizations
import plotly.graph_objects as go # for data visualization
import plotly.express as px # for data visualization

import warnings
warnings.filterwarnings("ignore")

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
df = pd.read_csv('drive/MyDrive/Ironhack/Data/Real estate.csv', encoding='utf-8')

In [53]:
X=df[['X3 distance to the nearest MRT station']]
y=df['Y house price of unit area']


In [54]:
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, random_state = 0)

In [55]:
# ------- Linear regression -------
model1 = LinearRegression()
lr = model1.fit(X_train, y_train)

# ------- Support Vector regression -------
model2 = SVR(kernel='rbf', C=100, epsilon=0.01) # set kernel and hyperparameters
svr = model2.fit(X_train, y_train)


In [56]:
print("MAE: ", metrics.mean_absolute_error(y_train, model2.predict(X_train)).round(4))
print("MSE: ", metrics.mean_squared_error(y_train, model2.predict(X_train)).round(4))
print("RMSE: ", np.sqrt(metrics.mean_squared_error(y_train, model2.predict(X_train))).round(4))
print("MAPE: ", metrics.mean_absolute_percentage_error(y_train, model2.predict(X_train)).round(4))
print("R2: ", metrics.r2_score(y_train, model2.predict(X_train)).round(4))

MAE:  6.1976
MSE:  84.2401
RMSE:  9.1782
MAPE:  0.1838
R2:  0.5506


In [57]:
print("MAE: ", metrics.mean_absolute_error(y_test, model2.predict(X_test)).round(4))
print("MSE: ", metrics.mean_squared_error(y_test, model2.predict(X_test)).round(4))
print("RMSE: ", np.sqrt(metrics.mean_squared_error(y_test, model2.predict(X_test))).round(4))
print("MAPE: ", metrics.mean_absolute_percentage_error(y_test, model2.predict(X_test)).round(4))
print("R2: ", metrics.r2_score(y_test, model2.predict(X_test)).round(4))

MAE:  5.6131
MSE:  57.7862
RMSE:  7.6017
MAPE:  0.161
R2:  0.6673


In [58]:
def plot(df, model1, model2):
  # Create a scatter plot
  fig = px.scatter(df, x=df['X3 distance to the nearest MRT station'], y=df['Y house price of unit area'],
                  opacity=0.8, color_discrete_sequence=['black'])
  X = df['X3 distance to the nearest MRT station']
  # Predict y values for our set of X values
  x_range = np.linspace(X.min(), X.max(), 100)
  y_lr = model1.predict(x_range.reshape(-1, 1)) # Linear regression
  y_svr = model2.predict(x_range.reshape(-1, 1)) # SVR

  # Add a best-fit line
  fig.add_traces(go.Scatter(x=x_range, y=y_lr, name='Linear Regression', line=dict(color='limegreen')))
  fig.add_traces(go.Scatter(x=x_range, y=y_svr, name='Support Vector Regression', line=dict(color='red')))
  fig.add_traces(go.Scatter(x=x_range, y=y_svr+10, name='+epsilon', line=dict(color='red', dash='dot')))
  fig.add_traces(go.Scatter(x=x_range, y=y_svr-10, name='-epsilon', line=dict(color='red', dash='dot')))

  # Change chart background color
  fig.update_layout(dict(plot_bgcolor = 'white'))

  # Update axes lines
  fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey',
                  zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey',
                  showline=True, linewidth=1, linecolor='black')

  fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey',
                  zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey',
                  showline=True, linewidth=1, linecolor='black')

  # Set figure title
  fig.update_layout(title=dict(text="House Price Based on Distance from the Nearest MRT",
                              font=dict(color='black')))
  # Update marker size
  fig.update_traces(marker=dict(size=3))

  fig.show()

plot(df, model1, model2)

In [72]:
X=df[['X3 distance to the nearest MRT station']]
y=df['Y house price of unit area']

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, random_state = 0)

In [74]:
from sklearn.model_selection import GridSearchCV

# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}

grid = GridSearchCV(SVR(), param_grid, refit = True, verbose = 3)

# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.013 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.004 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.017 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.011 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.007 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.016 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.006 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.021 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.015 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.013 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.032 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [75]:

print(grid.best_params_)

print(grid.best_estimator_)

{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
SVR(C=100, gamma=0.001)


In [80]:
print("MAE: ", metrics.mean_absolute_error(y_train, grid.predict(X_train)).round(4))
print("MSE: ", metrics.mean_squared_error(y_train, grid.predict(X_train)).round(4))
print("RMSE: ", np.sqrt(metrics.mean_squared_error(y_train, grid.predict(X_train))).round(4))
print("MAPE: ", metrics.mean_absolute_percentage_error(y_train, grid.predict(X_train)).round(4))
print("R2: ", metrics.r2_score(y_train, grid.predict(X_train)).round(4))

MAE:  3.9934
MSE:  55.6124
RMSE:  7.4574
MAPE:  0.1107
R2:  0.7033


In [81]:
def plot(df, grid):
  # Create a scatter plot
  fig = px.scatter(df, x=df['X3 distance to the nearest MRT station'], y=df['Y house price of unit area'],
                  opacity=0.8, color_discrete_sequence=['black'])
  X = df['X3 distance to the nearest MRT station']
  # Predict y values for our set of X values
  x_range = np.linspace(X.min(), X.max(), 100)
  y_svr = grid.predict(x_range.reshape(-1, 1)) # SVR

  # Add a best-fit line
  fig.add_traces(go.Scatter(x=x_range, y=y_svr, name='Support Vector Regression', line=dict(color='red')))
  fig.add_traces(go.Scatter(x=x_range, y=y_svr+10, name='+epsilon', line=dict(color='red', dash='dot')))
  fig.add_traces(go.Scatter(x=x_range, y=y_svr-10, name='-epsilon', line=dict(color='red', dash='dot')))

  # Change chart background color
  fig.update_layout(dict(plot_bgcolor = 'white'))

  # Update axes lines
  fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey',
                  zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey',
                  showline=True, linewidth=1, linecolor='black')

  fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey',
                  zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey',
                  showline=True, linewidth=1, linecolor='black')

  # Set figure title
  fig.update_layout(title=dict(text="House Price Based on Distance from the Nearest MRT",
                              font=dict(color='black')))
  # Update marker size
  fig.update_traces(marker=dict(size=3))

  fig.show()

plot(df, grid)